In [ ]:
import numpy as np
from fooof import FOOOF
from importlib import reload
import os
import mne
from matplotlib import pyplot as plt
# import own functions
from utils import find_folders
import fooof_ftg_fun
import json
import pandas as pd

### Add Directories/ Load Files

In [ ]:
reload(find_folders)
onedrive = find_folders.get_onedrive_path()
project_path = find_folders.get_onedrive_path("entrainment")
print(project_path)

In [ ]:
fft_np = np.load(os.path.join(
    project_path,
        'data',
        'FFTs',
        'without_med_FTG',
        'Sub050_FFT_ARTREJECTED.npy'
    )
)

SUBID = 'SUB_050'
ON = 1
OFF = 30
SIDE = 0

spectrum = np.mean(fft_np[SIDE,:,ON:OFF],1)

In [ ]:
plt.plot(spectrum)

In [ ]:
reload(fooof_ftg_fun)
PATH = os.path.join(project_path, 'results', 'FOOOF_FIT')
fm = fooof_ftg_fun.fit_fooof(SUBID, PATH, spectrum)

fm.print_results()

### Combine Jsons for each group

In [ ]:
json_filenames

In [ ]:

json_dir = os.path.join(project_path, 'results', 'FOOOF_FIT', 'without_med_FTG_allPS')

json_files = [f for f in os.listdir(json_dir) if f.endswith(".json")]

data_list = []

for file_name in json_files:
    file_path = os.path.join(json_dir, file_name)
    with open(file_path, 'r') as file:
        data = json.load(file)
        if 'r_squared_' in data and 'error_' in data and 'aperiodic_params_' in data:
            # Extract "key1" and "key2"
            extracted_data = {
                'PerceptID': file_name[0:7],
                'FTG_OFF': 0,
                # Extract the elements from the "list_key" list
                'aperiodic_offset': data['aperiodic_params_'][0] if len(data['aperiodic_params_']) > 0 else None,
                'aperiodic_exponent': data['aperiodic_params_'][1] if len(data['aperiodic_params_']) > 1 else None,
                'r_squared': data['r_squared_'],
                'error': data['error_']
            }
            data_list.append(extracted_data)

# Create a DataFrame from the extracted data
df = pd.DataFrame(data_list)

In [ ]:
df

In [ ]:
df.to_csv(os.path.join(json_dir,'Foof_Aperiodics_withoutMedFTG_allPS.csv'), index = False)

In [ ]:
#Load the two csvs
fooof_with = pd.read_csv(os.path.join(
    project_path, 'results', 'FOOOF_FIT', 'Foof_Aperiodics_withMedFTG.csv'
))

fooof_without = pd.read_csv(os.path.join(
    project_path, 'results', 'FOOOF_FIT', 'Foof_Aperiodics_withoutMedFTG.csv'
))

fooof_without


In [ ]:
#Make boxplots with the offset and the exponent
%matplotlib qt

fig, axes = plt.subplots(1, 2, figsize=(10, 5))
plt.subplots_adjust(wspace=0.5) 
# Aperiodic Offset
axes[0].boxplot([fooof_with['aperiodic_offset'], fooof_without['aperiodic_offset']], widths = 0.5, showfliers=False)
axes[0].scatter([1]*len(fooof_with['aperiodic_offset']), fooof_with['aperiodic_offset'], color='dimgrey', s =20)
axes[0].scatter([2]*len(fooof_without['aperiodic_offset']), fooof_without['aperiodic_offset'], color='darkred', s=20)
axes[0].set_xticklabels(['FTG DBS Off', 'No FTG DBS Off'])
axes[0].set_ylabel('Aperiodic Offset')

# Aperiodic Exponent
axes[1].boxplot([fooof_with['aperiodic_exponent'], fooof_without['aperiodic_exponent']], widths = 0.5, showfliers=False)
axes[1].scatter([1]*len(fooof_with['aperiodic_exponent']), fooof_with['aperiodic_exponent'], color='dimgrey', s =20)
axes[1].scatter([2]*len(fooof_without['aperiodic_exponent']), fooof_without['aperiodic_exponent'], color='darkred', s=20)
axes[1].set_xticklabels(['FTG DBS Off', 'No FTG DBS Off'])
axes[1].set_ylabel('Aperiodic Exponent')

In [ ]:
plt.savefig(os.path.join(
    project_path, 'results', 'FOOOF_FIT', 'Boxplots_Aperiodic'
), dpi = 400
)

In [ ]:
from scipy.stats import mannwhitneyu
statistic1, p_value1 = mannwhitneyu(fooof_with['aperiodic_offset'], fooof_without['aperiodic_offset'])
p_value1

In [ ]:
statistic2, p_value2 = mannwhitneyu(fooof_with['aperiodic_exponent'], fooof_without['aperiodic_exponent'])
p_value2